In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Medchem

Medchem is a package for applying general filtering rules on a set of molecules to ensure they have drug-like properties.

In this tutorial, we will apply various filtering on an example dataset to get highlight the package API

### Setup

In [95]:
import datamol as dm
import random
import numpy as np
from loguru import logger

data = dm.data.freesolv().sample(500)
smiles_list = data.smiles.values

In [96]:
from medchem.filter import lead
from medchem.demerits import score, batch_score
from medchem.novartis import NovartisFilters
from medchem.alerts import AlertFilters

### Using the filter module

The filter module provides a variety of two types of filters:
-  `generic`: custom filtering based on some given molecule properties such as number of atoms, presence of specific atom type, etc
-  `lead`: filtering based on structural motifs that are known to either be toxic, reactive, unstable or frequent false positive

In [97]:
# common filters including pains, brenk, nih, zinc
lead.common_filter(smiles_list, brenk=True, nih=True)

array([ True, False, False,  True, False, False, False,  True,  True,
       False,  True,  True,  True, False,  True, False,  True,  True,
        True, False,  True, False,  True, False, False, False,  True,
        True, False,  True,  True, False, False, False, False, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,  True,  True, False,  True,  True,  True, False,
        True,  True, False,  True,  True,  True, False, False,  True,
        True, False, False,  True, False,  True,  True, False,  True,
        True, False, False,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True, False, False,  True, False, False,  True,  True,
       False, False,  True,  True,  True, False,  True,  True, False,
       False,  True,  True, False, False, False,  True, False,  True,
        True, False,

In [98]:
# filtering based on some commons alerts + additional lead like rules
lead.alert_filter(smiles_list, alerts=["Glaxo", "BMS"], rule_dict=dict(MW=[0, 100]))

array([ True,  True, False,  True, False, False, False,  True, False,
       False, False,  True, False, False,  True, False, False, False,
       False,  True,  True, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False,  True,  True,  True,
       False, False,  True, False, False, False, False, False,  True,
       False,  True,  True, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False,  True,
       False, False,

In [99]:
# filtering based on NIBR screening deck process described in
# "Evolution of Novartis' small molecule screening deck design" by Schuffenhauer, A. et al. J. Med. Chem. (2020),
# https://dx.doi.org/10.1021/acs.jmedchem.0c01332.
lead.screening_filter(smiles_list, return_idx=True)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        14,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  47,  48,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 129, 130, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 168, 169,
       170, 171, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 193, 195, 196, 19

In [100]:
# Filter based on the demerit scoring of Eli Lilly
test_config = {
    "output": "test",
    "min_atoms": 10,
    "soft_max_atoms": 30,
    "hard_max_atoms": 50,
    "smarts": [],
    "nodemerit": False,
    "dthresh": 160,
    "odm": [],
    "okiso": False,
    "noapdm": False,
}
lead.demerit_filter(smiles_list, max_demerits=160, return_idx=True, **test_config)

array([  6,  10,  22,  33,  39,  50,  51,  74,  84, 107, 110, 123, 133,
       143, 153, 162, 163, 168, 172, 190, 200, 201, 203, 205, 206, 212,
       232, 254, 255, 286, 289, 294, 307, 309, 311, 315, 316, 317, 320,
       334, 344, 354, 357, 358, 360, 365, 368, 384, 391, 415, 426, 427,
       434, 435, 436, 444, 445, 446, 477, 481, 483, 485, 489, 492, 493])

### Advanced options

The advanced options allow a better control over the filtering process. They also provide more information on the issues with the molecules.

#### AlertFilter

These are the underlying filters called by `lead.alert_filter`. In the output, the compound status is indicated as either `"Exclude"` or `"Ok"`.

In [101]:
filter_obj = AlertFilters(alerts_set=["inpharmatica", "SureChEMBL"])
out = filter_obj(smiles_list)
out

,_smiles,status,reasons,MW,LogP,HBD,HBA,TPSA
0,Cn1cccc1,Ok,None,81.118,1.02510,0,1,4.93
1,CC=C(C)C,Ok,None,70.135,1.97250,0,0,0.00
2,Clc1ccccn1,Exclude,"halo-pyridine,_-diazoles_and_-triazoles; Filte...",113.547,1.73500,0,1,12.89
3,CCCCO,Ok,None,74.123,0.77880,1,1,20.23
4,CNC(=O)O/N=C\C(C)(C)SC,Exclude,Nitrosone_not_nitro; Filter18_oxime_ester; Fil...,190.268,1.46980,1,4,50.69
...,...,...,...,...,...,...,...,...
495,CCCCC#N,Ok,None,83.134,1.70018,0,1,23.79
496,FC(F)(F)c1ccccc1C(F)(F)F,Ok,None,214.108,3.72420,0,0,0.00
497,CCOC=O,Exclude,formate_formide; Filter38_aldehyde,74.079,0.17930,0,2,26.30
498,O=Cc1ccc(O)cc1,Exclude,Aldehyde; Filter38_aldehyde,122.123,1.20470,1,2,37.30


#### NovartisFilter

These are the underlying filters called by `lead.screening_filter`. 

Here is an explanation of the output:
- **status**: one of `["Exclude", "Flag", "Annotations", "Ok"]` (ordered by quality). Generally, you can keep anything without the "Exclude" label, as long as you also apply a maximum severity score for compounds that collects too many flags.
- **covalent**: number of potentially covalent motifs contained in the compound
- **severity**: how severe are the issues with the molecules:
      - `0`: compound has no flags, might have annotations;
      - `1-9`:  number of flags the compound raises;
      - `>= 10`:  default exclusion criterion used in the paper
- **special_mol**: whether the compound/parts of the compound belongs to a special class of molecules (e.g peptides, glycosides, fatty acid). In that case, you should review the rejection reasons.

In [102]:
filter_obj = NovartisFilters()
out = filter_obj(smiles_list)
out

,_smiles,status,reasons,severity,covalent,special_mol
0,Cn1cccc1,Ok,None,0,NaN,NaN
1,CC=C(C)C,Ok,None,0,NaN,NaN
2,Clc1ccccn1,Ok,None,0,NaN,NaN
3,CCCCO,Ok,None,0,NaN,NaN
4,CNC(=O)O/N=C\C(C)(C)SC,Flag,acetal_aminal_acyclic_min(1); oxim_ester_min(1),1,0.0,0.0
...,...,...,...,...,...,...
495,CCCCC#N,Ok,None,0,NaN,NaN
496,FC(F)(F)c1ccccc1C(F)(F)F,Ok,None,0,NaN,NaN
497,CCOC=O,Ok,None,0,NaN,NaN
498,O=Cc1ccc(O)cc1,Annotations,aldehyde_aromatic_min(1),0,1.0,0.0


#### Demerits scoring

Demerit scoring uses the Eli Lilly filter rules. Those are complex rules, that can be customized in any way you wish. 

The following "information" will be computed and added as columns to a DataFrame for each run:

- **status**: this was added for compatibility and has values `"Exclude"`, `"Flag"` or `"Ok"`.
- **rejected** : whether the molecule pass the filter or was rejected
- **reasons**: the reasons why the molecule was rejected if available
- **demerit_score** a demerit score for molecules. The lower the better. A cutoff is used to reject molecule with too many demerits, which you can refilter again after.
- **step**: step of the pipeline where molecule was filtered out, if available



In [103]:
out = score(smiles_list, **test_config)
out

,_smiles,ID,reasons,step,rejected,demerit_score,status
0,CN1C=CC=C1,0,not_enough_atoms,1,True,NaN,Exclude
1,CC=C(C)C,1,not_enough_atoms,1,True,NaN,Exclude
2,C1=CC=NC(=C1)Cl,2,not_enough_atoms,1,True,NaN,Exclude
3,CCCCO,3,not_enough_atoms,1,True,NaN,Exclude
4,CC(C)(/C=N\OC(=O)NC)SC,4,activated_ester,2,True,NaN,Exclude
...,...,...,...,...,...,...,...
495,CCCCC#N,495,not_enough_atoms,1,True,NaN,Exclude
496,C1=CC=C(C(=C1)C(F)(F)F)C(F)(F)F,496,no_interesting_atoms,1,True,NaN,Exclude
497,CCOC=O,497,not_enough_atoms,1,True,NaN,Exclude
498,C1=CC(=CC=C1C=O)O,498,not_enough_atoms,1,True,NaN,Exclude


In [104]:
# Although the demirits.score is already quite fast, you can also call the parallelized version of it using the `batch_score` function

In [134]:
out2 = batch_score(smiles_list, n_jobs=2,  batch_size=100,  progress=True, **test_config)
out2

100%|██████████| 5/5 [00:00<00:00, 11.95it/s]


,_smiles,ID,reasons,step,rejected,demerit_score,status
0,CN1C=CC=C1,0,not_enough_atoms,1,True,NaN,Exclude
1,CC=C(C)C,1,not_enough_atoms,1,True,NaN,Exclude
2,C1=CC=NC(=C1)Cl,2,not_enough_atoms,1,True,NaN,Exclude
3,CCCCO,3,not_enough_atoms,1,True,NaN,Exclude
4,CC(C)(/C=N\OC(=O)NC)SC,4,activated_ester,2,True,NaN,Exclude
...,...,...,...,...,...,...,...
495,CCCCC#N,495,not_enough_atoms,1,True,NaN,Exclude
496,C1=CC=C(C(=C1)C(F)(F)F)C(F)(F)F,496,no_interesting_atoms,1,True,NaN,Exclude
497,CCOC=O,497,not_enough_atoms,1,True,NaN,Exclude
498,C1=CC(=CC=C1C=O)O,498,not_enough_atoms,1,True,NaN,Exclude
